In [33]:
import pandas as pd

In [160]:
# reads in esposts-data dataframes
# starts all teams elos at 1500

teams_df = pd.read_json("./esports-data/teams.json")
teams_df["elo"] = 1500.00

mapping_df = pd.read_json("./esports-data/mapping_data.json")

In [ ]:
# reads in all game events, and finds last stats_update for end of game stats

game_df = pd.read_json("./games/ESPORTSTMNT03_3196037.json")
end_of_game_stats = game_df[game_df["eventType"] == "stats_update"].drop_duplicates(subset=["eventType"], keep="last")["participants"].values[0]
game_end = game_df[game_df["eventType"] == "game_end"]

In [140]:
# determines winning team, losing team, team gold, and team acronym

winning_team = int(game_end["winningTeam"].values[0])
if winning_team == 100:
    losing_team = 200
else:
    losing_team = 100

winning_team_prefix = ""
losing_team_prefix = ""

win_side_gold = 0
lose_side_gold = 0

for player in end_of_game_stats:
    if player["teamID"] == winning_team:
        win_side_gold += player["totalGold"]
        winning_team_prefix = player["playerName"].split(' ')[0]
    else:
        lose_side_gold += player["totalGold"]
        losing_team_prefix = player["playerName"].split(' ')[0]

win_side_gold, lose_side_gold, winning_team_prefix, losing_team_prefix

(53655, 41352, 'MIR', 'ROTM')

In [91]:
# platformGameId for the game played

platformGameId = game_df[game_df["eventType"] == "game_info"]["platformGameId"].values[0]
platformGameId

'ESPORTSTMNT03:3196037'

In [142]:
# game_df team ids -> teams_df ids
# 109981650516317055 -> 109981650516317056
# 110733889090164854 -> 110733889090164848
# ids are unreliable, use acronyms in usernames maybe?
winning_team = teams_df[teams_df["acronym"] == winning_team_prefix]
losing_team = teams_df[teams_df["acronym"] == losing_team_prefix]

In [145]:
# pulls the current elo of the teams (starts at 1500 by default)

winning_team_elo = winning_team["elo"].values[0]
losing_team_elo = losing_team["elo"].values[0]

winning_team_elo, losing_team_elo

(1500, 1500)

In [161]:
# elo calculation, in future actual_performance should be found via a model and k needs to be adjusted accordingly
# actual_performance is [0,1] and should account for a winning team's performance (not probability to win!)
# the losing team's performance is 1 - actual_performance
# thoughts on actual_performance being floored at a certain val? (depending on model. currently .55)

expected_performance_win = 1 / (1 + pow(10, (winning_team_elo - losing_team_elo) / 400))
expected_performance_lose = 1 / (1 + pow(10, (winning_team_elo - losing_team_elo) / 400))
actual_performace = max(.55, .5 + (win_side_gold - lose_side_gold) / (win_side_gold + lose_side_gold))
k = 100

new_elo_win = winning_team_elo + k * (actual_performace - expected_performance_win)
new_elo_lose = losing_team_elo + k * ((1 - actual_performace) - expected_performance_lose)

new_elo_win, new_elo_lose

In [165]:
# updates elo of teams in teams_df to new values

teams_df.loc[teams_df["acronym"] == winning_team_prefix, "elo"] = new_elo_win
teams_df.loc[teams_df["acronym"] == losing_team_prefix, "elo"] = new_elo_lose
teams_df[teams_df["elo"] != 1500]

,team_id,name,acronym,slug,elo
225,109981650516317056,Mirage Alliance,MIR,mirage-alliance,1512.949572
231,110733889090164848,Return of the Middlesticks,ROTM,return-of-the-middlesticks,1487.050428
